## GPS Processing Code 1

This code takes individual GPS time series data from individual stations and:

1. removes outliers (all points that are >50 mm away from neighboring points)
2. removes steps 
3. smooths the time series (applies a 30-day moving average)
4. removies annual and biannual seasonal signals and detrends the entire time series

This code requires a step file with the format:

      Station_name ------ YYYYMMDD

The format of the GPS file should follow:

      YYYYMMDD ------ dN ------ dE ------ dU ------ sigmaN ------ sigmaE ------ sigmaU

In [1]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from datetime import timedelta
from datetime import datetime as dt
import time
import numpy as np

In [2]:
def find_outliers(diff):

    outlier_idx = diff.index[diff['N'].abs()>50].to_list()
    outlier_idx.extend(diff.index[diff['E'].abs()>50].to_list())
    outlier_idx.extend(diff.index[diff['U'].abs()>50].to_list())
    outlier_idx.sort()
    
    return outlier_idx
    

In [3]:
def remove_outliers(GPS_def,outlier_idx):
# Takes the indices of outliers and replaces them in the data frame by either
# 1) averaing the data points on either side or
# 2) setting the outlier value equal to the value 1 day before it. This is done in case there are 
# consecutive days of outliers and averaging is not appropriate
    
    fixed = [] #keep track of dates so outliers aren't removed twice

    for idx, val in enumerate(outlier_idx):
    
        if fixed.count(val) == 0:
            if outlier_idx.count(idx+1) > 0: #option 2
                GPS_df.loc[val,'N'] = GPS_df.loc[val-1,'N']
                GPS_df.loc[val,'E'] = GPS_df.loc[val-1,'E']
                GPS_df.loc[val,'U'] = GPS_df.loc[val-1,'U']
            else: #option 1
                GPS_df.loc[val,'N'] = (GPS_df.loc[val-1,'N'] + GPS_df.loc[val+1,'N'])/2
                GPS_df.loc[val,'E'] = (GPS_df.loc[val-1,'E'] + GPS_df.loc[val+1,'E'])/2
                GPS_df.loc[val,'U'] = (GPS_df.loc[val-1,'U'] + GPS_df.loc[val+1,'U'])/2
        fixed.append(outlier_idx[idx])

In [4]:
def remove_steps(GPS_df, station_steps, comp):
# Takes step dates for a given station and component and:
# 1) finds their corresponding date in the Dataframe
# 2) averages the positions 21 days before and after that date
# 3) calculates the difference of these two averages as the offset (after - before)
# 4) subtracts this offset from all dates after the step date

    station_steps = station_steps.reset_index(drop=True)
    
    for index, row in station_steps.iterrows():
        idx = GPS_df.index[GPS_df['Date']==station_steps['Date'].iloc[index]].to_list()
        
        #if dataframe does not contain the step date, then step back one day at a time
        #to use the closest day before the step date that is contained in the dataframe
        shift=1
        while not idx:
            idx = GPS_df.index[GPS_df['Date']==station_steps['Date'].iloc[index]-timedelta(days=shift)].to_list()
            shift += 1
        
        #average position before event
        if idx[0] < 21: #if step date falls within 21 days of the start of the time series
            before = GPS_df[comp].iloc[:idx[0]-1].mean()
        else:
            before = GPS_df[comp].iloc[idx[0]-21:idx[0]-1].mean()
        #average position after event
        if idx[0] + 21 > len(GPS_df.index): #if step date falls within 21 days of the end of the time series
            after = GPS_df[comp].iloc[idx[0]+1:].mean()
        else:
            after = GPS_df[comp].iloc[idx[0]+1:idx[0]+21].mean()
        offset = after - before

        GPS_df.loc[GPS_df.index >= idx[0],comp] -= offset

In [5]:
def toYearFraction(date):
# Calculates the decimal year for a given datetime object
    
    def sinceEpoch(date): #returns seconds since epoch
        return time.mktime(date.timetuple())
    s = sinceEpoch
    
    year = date.year
    startOfThisYear = dt(year = year, month=1, day=1)
    startOfNextYear = dt(year = year+1, month=1, day=1)
    
    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration
    
    return date.year + fraction

In [6]:
def remove_seasonal(GPS_df,comp,dates):
#Removes the annual and biannual seasonal compenents and detrends the entire timeseries.
    
    A = np.array([np.sin(2 * np.pi * dates), np.cos(2 * np.pi * dates),
              np.sin(4 * np.pi * dates), np.cos(4 * np.pi * dates), dates, np.ones((dates.size))])
    A = np.transpose(A)

    data = GPS_df[comp].to_numpy()
    data = data.reshape(-1,1)

    coeff = np.linalg.lstsq(A, data, rcond=None)
 
    newData = data - np.matmul(A, coeff[0])
    GPS_df[comp] = newData


In [ ]:
files = glob.glob('????')

for file in files:
    
    GPS_df = pd.read_csv(file, delim_whitespace=True, names=['Date','N','E','U','sN','sE','sU'])
    GPS_df['Date'] = pd.to_datetime(GPS_df['Date'], format='%Y%m%d')

    fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize =(15,10))
    ax1.plot(GPS_df['Date'],GPS_df['N'])
    ax1.set_title(file + ' North')
    ax2.plot(GPS_df['Date'],GPS_df['E'])
    ax2.set_title(file + ' East')
    ax2.set_ylabel('Displacement')
    ax3.plot(GPS_df['Date'],GPS_df['U'])
    ax3.set_title(file + ' Vertical')
    ax3.set_xlabel('Time')
    
#remove outliers
    
    df_diff = GPS_df.diff()
    outlier_idx = find_outliers(df_diff)
    
    while outlier_idx:
        remove_outliers(GPS_df, outlier_idx)
        df_diff = GPS_df.diff()
        outlier_idx = find_outliers(df_diff)
    
    ax1.plot(GPS_df['Date'],GPS_df['N'],'b')
    ax2.plot(GPS_df['Date'],GPS_df['E'],'b')
    ax3.plot(GPS_df['Date'],GPS_df['U'],'b')
    
#remove steps
    
    step_df = pd.read_csv('step_file.txt', delim_whitespace=True, names=['Station','Date'])
    step_df['Date'] = pd.to_datetime(step_df['Date'], format='%Y%m%d')
    
    station_steps = step_df[step_df['Station'].str.match(file)]
    
    if not station_steps.empty:
        if file == 'P159':  #optional caveat for station P159, correction for the 
                            #the 2020 EQ disrupts the end of the time series on the N component
            remove_steps(GPS_df,station_steps[-1:],'N')
        else:
            remove_steps(GPS_df,station_steps,'N')
        remove_steps(GPS_df,station_steps,'E')
        remove_steps(GPS_df,station_steps,'U')

    ax1.plot(GPS_df['Date'],GPS_df['N'],'r')
    ax2.plot(GPS_df['Date'],GPS_df['E'],'r')
    ax3.plot(GPS_df['Date'],GPS_df['U'],'r')
    
#apply 30-day moving average

    GPS_df['N']=GPS_df['N'].rolling(30).mean()
    GPS_df['E']=GPS_df['E'].rolling(30).mean()
    GPS_df['U']=GPS_df['U'].rolling(30).mean()

    GPS_df = GPS_df.dropna(axis=0)
    GPS_df = GPS_df.reset_index(drop=True)
    
    ax1.plot(GPS_df['Date'],GPS_df['N'],'k')
    ax2.plot(GPS_df['Date'],GPS_df['E'],'k')
    ax3.plot(GPS_df['Date'],GPS_df['U'],'k')

#remove annual + biannual + trend components

    dates = np.zeros(len(GPS_df))

    for index, row in GPS_df.iterrows():
        dates[index] = toYearFraction(GPS_df['Date'].iloc[index])
        
    remove_seasonal(GPS_df,'N', dates)
    remove_seasonal(GPS_df,'E', dates)
    remove_seasonal(GPS_df,'U', dates)
    
    ax1.plot(GPS_df['Date'],GPS_df['N'],'g')
    ax2.plot(GPS_df['Date'],GPS_df['E'],'g')
    ax3.plot(GPS_df['Date'],GPS_df['U'],'g')

    fig, (ax4, ax5, ax6) = plt.subplots(3,1, figsize=(15,15))
    
    ax4.plot(GPS_df['Date'],GPS_df['N'],'b.')
    ax4.set_title(file + ' Final North')
    ax5.plot(GPS_df['Date'],GPS_df['E'],'b.')
    ax5.set_title(file + ' Final East')
    ax6.plot(GPS_df['Date'],GPS_df['U'],'b.')
    ax6.set_title(file + ' Final Up')
    
    #plt.show()      #option to show each step

    #input("Enter")  #option to have the code pause for each station and require the user to press "enter" to continue
    #clear_output()
    
    plt.close('all')
    
    file_name = file +"_cleaned.txt"
    
    np.savetxt(file_name, GPS_df.values, fmt='%s')
    